<div style="text-align:center; background-color:blue; padding:10px;">
  <h1 style="color:white;">EDA</h1>
</div>
<br>

<div style="text-align:left; background-color:gray; padding:0px;">
  <h1 style="color:white;">Import des librairies</h1>
</div>
<br>

In [1]:
# Visualisation de données
import pandas as pd
pd.set_option('display.max_columns', None)
import seaborn as sns

# Librairies pour la connection à rds
from connection import connection_with_sqlalchemy

<div style="text-align:left; background-color:gray; padding:0px;">
  <h1 style="color:white;">Récupération des données sur RDS</h1>
</div>
<br>

In [4]:
#///////////////////////////////////////////////////////////////////////////////
#           A FAIRE UNE SEULE FOIS LORS DU LANCEMENT DU NOTEBOOK
#///////////////////////////////////////////////////////////////////////////////

In [2]:
# Utilisation de sqlalchemy pour charger les données datagouv dans RDS
# Création de la connection

# engine = connection_with_sqlalchemy("datagouv")

# query="""
# SELECT 
#     V.*,
#     T.NAME_TYPE_BIEN,
#     C.NAME_COMMUNE,
#     D.Name_departement,
#     R.Name_region
# FROM VENTES V
# INNER JOIN TYPES_BIENS as T ON V.ID_TYPE_BIEN = T.ID_TYPE_BIEN
# INNER JOIN COMMUNES AS C ON V.ID_COMMUNE = C.ID_COMMUNE
# INNER JOIN DEPARTEMENTS AS D ON C.ID_DEPT = D.ID_DEPT
# INNER JOIN REGIONS R ON D.ID_REGION = R.ID_REGION;
# """
# df = pd.read_sql(query, engine)
# df.head(10)

,ID_VENTE,MONTANT,NUMERO_RUE,RUE,CODE_POSTAL,LONGITUDE,LATITUDE,DATE_MUTATION,SURFACE_BATI,NB_PIECES,SURFACE_TERRAIN,DEPENDANCES,ID_TYPE_BIEN,ID_COMMUNE,NAME_TYPE_BIEN,NAME_COMMUNE,Name_departement,Name_region
0,2,37000,5,RUE D YPRES,01000,5.220403,46.197326,2018-07-06,30,1,0,1,1,01053,Appartement,Bourg-en-Bresse,Ain,Auvergne-Rhône-Alpes
1,43,74000,5670,BD MORINET,16260,0.450180,45.825141,2018-08-02,32,2,621,1,1,16085,Appartement,Chasseneuil-sur-Bonnieure,Charente,Nouvelle-Aquitaine
2,48,50000,13,GR GRANDE RUE,16110,0.386824,45.740376,2018-08-01,113,5,0,1,1,16281,Appartement,La Rochefoucauld-en-Angoumois,Charente,Nouvelle-Aquitaine
3,87,45000,13,GR GRANDE RUE,16110,0.386824,45.740376,2018-08-09,103,3,0,1,1,16281,Appartement,La Rochefoucauld-en-Angoumois,Charente,Nouvelle-Aquitaine
4,118,120000,21,BD VICTOR HUGO,01000,5.229090,46.201418,2018-09-28,82,4,0,0,1,01053,Appartement,Bourg-en-Bresse,Ain,Auvergne-Rhône-Alpes
5,167,123000,2,RUE THEOPHILE GIBOUIN,16500,0.670092,46.015351,2018-08-31,48,3,2262,0,1,16106,Appartement,Confolens,Charente,Nouvelle-Aquitaine
6,179,254000,3,CHAMPS DE LA MONTEE,16510,0.225537,45.993701,2018-09-17,100,4,61625,0,1,16400,Appartement,Verteuil-sur-Charente,Charente,Nouvelle-Aquitaine
7,218,16000,41,RUE GABRIEL PERI,02100,3.283414,49.844160,2018-11-09,39,2,0,0,1,02691,Appartement,Saint-Quentin,Aisne,Hauts-de-France
8,233,60000,154,BD GAMBETTA,02100,3.292034,49.852991,2018-10-25,51,3,0,0,1,02691,Appartement,Saint-Quentin,Aisne,Hauts-de-France
9,265,47000,26,RUE DU VIEUX PORT,02100,3.277169,49.841145,2018-10-26,48,2,0,0,1,02691,Appartement,Saint-Quentin,Aisne,Hauts-de-France


In [ ]:
# Fermeture de la connection
# engine.close()

In [3]:
# Sauvegarde du data frame en csv pour éviter de requêter toute la table plusieurs fois
# df.to_csv('datas_rds.csv', index=False)

<div style="text-align:left; background-color:gray; padding:0px;">
  <h1 style="color:white;">Visualisation des données sur RDS</h1>
</div>
<br>

In [2]:
df = pd.read_csv("datas_rds.csv", low_memory=False)
df.head()

,ID_VENTE,MONTANT,NUMERO_RUE,RUE,CODE_POSTAL,LONGITUDE,LATITUDE,DATE_MUTATION,SURFACE_BATI,NB_PIECES,SURFACE_TERRAIN,DEPENDANCES,ID_TYPE_BIEN,ID_COMMUNE,NAME_TYPE_BIEN,NAME_COMMUNE,Name_departement,Name_region
0,2,37000,5,RUE D YPRES,1000,5.220403,46.197326,2018-07-06,30,1,0,1,1,01053,Appartement,Bourg-en-Bresse,Ain,Auvergne-Rhône-Alpes
1,43,74000,5670,BD MORINET,16260,0.450180,45.825141,2018-08-02,32,2,621,1,1,16085,Appartement,Chasseneuil-sur-Bonnieure,Charente,Nouvelle-Aquitaine
2,48,50000,13,GR GRANDE RUE,16110,0.386824,45.740376,2018-08-01,113,5,0,1,1,16281,Appartement,La Rochefoucauld-en-Angoumois,Charente,Nouvelle-Aquitaine
3,87,45000,13,GR GRANDE RUE,16110,0.386824,45.740376,2018-08-09,103,3,0,1,1,16281,Appartement,La Rochefoucauld-en-Angoumois,Charente,Nouvelle-Aquitaine
4,118,120000,21,BD VICTOR HUGO,1000,5.229090,46.201418,2018-09-28,82,4,0,0,1,01053,Appartement,Bourg-en-Bresse,Ain,Auvergne-Rhône-Alpes


In [3]:
print("Nombre de ventes dans le data frame : ", df.shape[0])

Nombre de ventes dans le data frame :  4205071


In [4]:
# Retrait des colonnes inutiles 
df = df.drop(columns=["ID_VENTE", "ID_TYPE_BIEN"])
# Recherche du nombre de lignes identiques
print(f"Il y a {df.duplicated().sum()} ligne(s) identique(s)")

Il y a 9 ligne(s) identique(s)


In [ ]:
# Suppression des lignes identiques
df = df.drop_duplicates()